In [1]:
# ====================== 环境准备 ======================
# 1) 设定环境变量（必须在导入 matplotlib 之前执行）
import os
from copy import deepcopy
from copy import deepcopy

os.environ['QT_API'] = 'pyqt5'        # 指定使用 PyQt5 作为 Qt 绑定
os.environ['MPLBACKEND'] = 'QtAgg'    # 指定 Matplotlib 后端为 QtAgg（更推荐，替代 TkAgg）

# 2) 启动 Qt 事件循环（控制台模式下，让 Qt 窗口能实时响应）
%gui qt5
from copy import deepcopy

# 3) 检查 matplotlib backend
import matplotlib as mpl
mpl.rcParams.update({
    # 这里按系统常见字体给一串候选，存在则自动生效
    "font.sans-serif": ["Microsoft YaHei", "SimHei", "SimSun",
                        "Noto Sans CJK SC", "Source Han Sans SC",
                        "Arial Unicode MS", "DejaVu Sans"],
    "font.family": "sans-serif",
    "axes.unicode_minus": False,   # 负号用正常字符，避免被当作缺字形
})

print("backend (before pyplot):", mpl.get_backend())
# 如果不是 QtAgg，强制改为 QtAgg（注意：必须在导入 pyplot 前设置）
mpl.rcParams['backend'] = 'QtAgg'

# 4) 现在再导入 pyplot
import matplotlib.pyplot as plt
print("backend (after pyplot):", mpl.get_backend())

# ====================== 导入依赖 ======================
import sys
# 避免反复执行时 Qt 类重复导入导致崩溃：如果已加载，先删除再导入
if 'draw.pyqt_draw.pyqt_main2' in sys.modules:
    del sys.modules['draw.pyqt_draw.pyqt_main2']

from PyQt5 import QtWidgets
import pyqtgraph as pg
from draw.pyqt_draw.pyqt_main2 import SatelliteViewer
import draw.read_snap_xml as read_snap_xml

# 配置 pyqtgraph：开启抗锯齿，关闭 OpenGL（更稳定）
pg.setConfigOptions(antialias=True)
# pg.setConfigOptions(useOpenGL=False)   # 若驱动或 OpenGL 有问题可显式关闭
# ====================== 基础参数 ======================
# 星座参数：每轨道卫星数 N，轨道平面数 P
N = 36
P = 18
import genaric2.tegnode as tegnode
import draw.basic_functio.write2xml as write2xml
from config import DATA_DIR,INPUT_DIR


backend (before pyplot): QtAgg
backend (after pyplot): qtagg


In [2]:
from draw.pyqt_draw.pyqt_main2 import SatelliteViewer


import genaric2.tegnode as tegnode

In [35]:
start1 =6814

end1 = 8485

file_path = INPUT_DIR / f"interplane_links_{start1}_{end1}.xml"
nodes1 = write2xml.xml_to_nodes(file_path, tegnode.tegnode)
# nodes1  = write2xml.xml_to_nodes(  rf"E:\研究生\研究进展\工作记录\实验记录\interplane_links_{start_ts}_{end_ts}.xml", tegnode.tegnode)


In [36]:
start2 =8485

end2 = 9355

file_path = INPUT_DIR / f"interplane_links_{start2}_{end2}.xml"
nodes2 = write2xml.xml_to_nodes(file_path, tegnode.tegnode)
# nodes2  = write2xml.xml_to_nodes(  rf"E:\研究生\研究进展\工作记录\实验记录\interplane_links_{start_ts}_{end_ts}.xml", tegnode.tegnode)
#
# nodes2  = write2xml.xml_to_nodes( = INPUT_DIR+'interplane_links_{start_ts}_{end_ts}.xml"', tegnode.tegnode)

In [37]:

start3 =9355

end3 = 10717

file_path = INPUT_DIR / f"interplane_links_{start3}_{end3}.xml"
nodes3 = write2xml.xml_to_nodes(file_path, tegnode.tegnode)
#
# nodes3  = write2xml.xml_to_nodes(  rf"E:\研究生\研究进展\工作记录\实验记录\interplane_links_{start_ts}_{end_ts}.xml", tegnode.tegnode)

In [38]:
totalnode_raw = {**nodes1, **nodes2,**nodes3 }


In [39]:
totalnode = deepcopy(totalnode_raw)

In [40]:
# ====================== 读取数据 ======================
# xml_file = r"E:\Data\station_visible_satellites_648_1d_real.xml"


xml_file = DATA_DIR / "station_visible_satellites_648_1d_real.xml"

# 解析 XML 得到 group_data，结构：{time_step: {'groups': {...}}}
group_data = read_snap_xml.parse_xml_group_data(xml_file, start1, end3)


In [ ]:
rev_group_data,offset = read_snap_xml.modify_group_data(group_data, N=36, groupid=4)

## 处理冲突边我们现在就需要将汇总的nodes，重新进行一次冲突边处理,我们一开始就是分而治之，所以，这里我们需要将汇总的nodes，重新进行一次冲突边处理

在进行操作的时候，一定要留意，我们代码中，有可能会修改totalnode内容的。下面的作用就是将总的totalnode转为边，这样我们就获得整个区间的情况

In [41]:
import draw.basic_functio.transnodes as transnodes
totalnode_comple = transnodes.transnodes(totalnode)

In [42]:
time_2_build=60
import draw.basic_functio.conflict_link as conflict_link
# 这个是目前最新的，仍然有小bug，但是暂时不修了，等后面再修
raw_edges_by_step,pending_edges,modifynodes = conflict_link.get_no_conflict_link_nodes3(totalnode_comple , start1, end3,time_2_build,N,P)


In [43]:

# 这里，我们需要将nodes 信息转化为edge信息,注意，这个edge也只有inter-edge，并且是原始
import draw.basic_functio.inter_edge2nodes as inter_edge2nodes

# all_inter_edge = inter_edge2nodes.trans_nodes2edges(totalnode,P,N)

all_inter_edge = raw_edges_by_step

In [29]:
# 转化为inter-edge信息后，我们可以通过绘图来查看是否是争取的

# ====================== 绘图初始化 ======================
# 1) QApplication 实例（全局唯一）
app = QtWidgets.QApplication.instance() or QtWidgets.QApplication([])

# 2) 确保 viewer 有全局引用，避免 GC 回收导致崩溃
if not hasattr(sys.modules[__name__], "_viewer_list"):
    _viewer_list = []



In [30]:

# 3) 创建并配置 viewer
viewer = SatelliteViewer(group_data)
viewer.setWindowTitle("onestep")
viewer.resize(1200, 700)
# viewer.edges_by_step = all_inter_edge
# viewer.pending_links_by_step =pending_edges

viewer.edges_by_step = raw_edges_by_step
viewer.pending_links_by_step =pending_edges
viewer.show()



肉眼检查后没问题，记录下一段区间。

In [44]:


start, end = start2, end2
filternodes = {}

for step in range(start, end):
    for x in range(P):
        for y in range(N):
            key = (x, y, step)
            if key in modifynodes:        # 只保留存在的键
                filternodes[key] = modifynodes[key]


再次检查

In [15]:

# 这里，我们需要将nodes 信息转化为edge信息,注意，这个edge也只有inter-edge，并且是原始
import draw.basic_functio.inter_edge2nodes as inter_edge2nodes
all_inter_edge = inter_edge2nodes.trans_nodes2edges(filternodes,P,N)
time_2_build = 60
# 这里，我们还需要将pending edges 信息也加入到all_inter_edge 中
pending_edges = inter_edge2nodes.trans_nodes2_pendingedges(filternodes, start1, end3, time_2_build,P, N)


In [16]:
# 转化为inter-edge信息后，我们可以通过绘图来查看是否是争取的

# ====================== 绘图初始化 ======================
# 1) QApplication 实例（全局唯一）
app = QtWidgets.QApplication.instance() or QtWidgets.QApplication([])

# 2) 确保 viewer 有全局引用，避免 GC 回收导致崩溃
if not hasattr(sys.modules[__name__], "_viewer_list"):
    _viewer_list = []


# 3) 创建并配置 viewer
viewer = SatelliteViewer(group_data)
viewer.setWindowTitle("flitter node grpah two-auth")
viewer.resize(1200, 700)
viewer.edges_by_step = all_inter_edge
viewer.pending_links_by_step =pending_edges
viewer.show()



检查无误后，存入到文件夹内

In [34]:

file_path = INPUT_DIR / f"onestep/interplane_links_{start}_{end}.xml"


write2xml.nodes_to_xml2(
    filternodes,
   file_path
)


In [45]:
start, end = start3, end3

filternodes = {}

for step in range(start, end):
    for x in range(P):
        for y in range(N):
            key = (x, y, step)
            if key in modifynodes:        # 只保留存在的键
                filternodes[key] = modifynodes[key]


In [46]:

file_path = INPUT_DIR / f"onestep/interplane_links_{start}_{end}.xml"


write2xml.nodes_to_xml2(
    filternodes,
   file_path
)